# Section 8: OpenQASM Operations

> **Exam Weight**: 6% (~4 Questions) | **Difficulty**: Medium | **Status**: EXAM TRAP - Static Methods!

## Learning Objectives
By the end of this notebook, you will be able to:
- Export circuits to QASM using `qc.qasm()` (instance method)
- Import circuits from QASM using `QuantumCircuit.from_qasm_str()` (static method!)
- Use `qiskit.qasm2` and `qiskit.qasm3` modules for QASM operations
- Understand QASM structure: header, registers, gates, measurements
- Avoid the most common exam trap: static vs instance methods

---

## 🧠 Conceptual Deep Dive

### The Recipe Card Analogy 📝

Think of **OpenQASM** as a **standardized recipe card format** for quantum cooking:

| Cooking | OpenQASM | Purpose |
|---------|----------|---------|
| **Your Dish** | Your circuit | The actual quantum program |
| **Write Recipe** | `qc.qasm()` | Export your circuit to text |
| **Recipe Library** | `QuantumCircuit` class | Where recipes are stored |
| **Get Recipe** | `.from_qasm_str()` | Load a recipe from the library |

```
The Recipe Card Workflow:
┌─────────────────────────────────────────────────────┐
│                                                     │
│  Export (Instance Method):                          │
│  ┌─────────┐      qc.qasm()       ┌─────────────┐  │
│  │ Circuit │ ──────────────────▶  │ QASM String │  │
│  └─────────┘                      └─────────────┘  │
│      ↑                                              │
│      │ You have a dish, write down the recipe      │
│                                                     │
│  Import (STATIC Method):                            │
│  ┌─────────────┐  QuantumCircuit.  ┌─────────┐    │
│  │ QASM String │ ─from_qasm_str()─▶│ Circuit │    │
│  └─────────────┘                   └─────────┘    │
│      ↑                                              │
│      │ Go to Kitchen (Class) to make from recipe   │
│                                                     │
└─────────────────────────────────────────────────────┘
```

### 🚨 THE STATIC METHOD TRAP (EXAM CRITICAL!)

```python
# ❌ WRONG - Most common exam mistake!
qc = QuantumCircuit(2)
qc.from_qasm_str(qasm_string)  # Instance method - FAILS!

# ✅ CORRECT - Static method!
qc = QuantumCircuit.from_qasm_str(qasm_string)
```

> **Memory Trick**: "**E**xport from **E**xisting, **I**mport to **I**nitialize"
> - Export = Existing circuit (`qc.qasm()`)
> - Import = Initialize new (`QuantumCircuit.from_qasm_str()`)

---

## Setup

## Part 1: Why OpenQASM Matters

### Why Learn This?

1. **Interoperability**: Share circuits between different quantum frameworks
2. **Persistence**: Save circuits as human-readable text
3. **Version Control**: Store circuits in git repositories
4. **Debugging**: Inspect circuit structure in text format

### The Mental Model

**Mental Model**: You don't ask a cake to create a new cake from a recipe. You go to the Kitchen (Class) to make a new cake!

```python
# Export: Instance method (makes sense!)
qasm_string = qc.qasm()  # "This circuit, give me your QASM"

# Import: STATIC method (counterintuitive!)
# WRONG: qc.from_qasm_str(string)  # Why would existing circuit import?
# RIGHT: QuantumCircuit.from_qasm_str(string)  # Class creates new circuit
```

## Part 2: Method Cheat Sheet (MEMORIZE!)

```python
# ═══════════════════════════════════════════════════════
#  EXPORT: Instance Method
# ═══════════════════════════════════════════════════════
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Export to QASM string (instance method - use the circuit)
qasm_string = qc.qasm()  # ✓ Instance method

# ═══════════════════════════════════════════════════════
#  IMPORT: STATIC Methods (CLASS level!)
# ═══════════════════════════════════════════════════════
# From string (STATIC - use the CLASS)
new_qc = QuantumCircuit.from_qasm_str(qasm_string)  # ✓ STATIC

# From file (STATIC - use the CLASS)
new_qc = QuantumCircuit.from_qasm_file('circuit.qasm')  # ✓ STATIC

# ═══════════════════════════════════════════════════════
#  COMMON MISTAKES (EXAM TRAPS!)
# ═══════════════════════════════════════════════════════
❌ qc.from_qasm_str(qasm_string)      # WRONG! Not instance method!
❌ from_qasm_str(qasm_string)         # WRONG! Need QuantumCircuit.
❌ QuantumCircuit.qasm()              # WRONG! qasm() is instance method!
✅ QuantumCircuit.from_qasm_str(s)   # CORRECT! Static method!
```

In [ ]:
# Imports for all examples
from qiskit import QuantumCircuit
from qiskit.qasm3 import dump, dumps, load, loads  # QASM 3.0 module
from qiskit import qasm3  # QASM 3.0 module

## Part 3: QASM Export - qasm3.dumps()

In [ ]:
def demonstrate_qasm_export():
    """
    QASM Export - Using qasm3.dumps()
    """
    print("\n" + "="*70)
    print("QASM 3.0 EXPORT")
    print("="*70)
    
    # Create circuit
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    print("\n[1] Circuit:")
    print(qc.draw())
    
    print("\n[2] Export with qasm3.dumps()")
    print("-" * 50)
    qasm_string = qasm3.dumps(qc)
    print("qasm_string = qasm3.dumps(qc)")
    print("\nResult:")
    print(qasm_string)
    
    print("\n🎯 EXAM TIP: Use qasm3.dumps(qc) for QASM 3.0 export!")
    print("   Returns OpenQASM 3.0 formatted string")

demonstrate_qasm_export()

---

## 8.2 QASM Import - qasm3.loads()

**EXAM QUESTION**: How do you create a circuit from a QASM 3.0 string?

**ANSWER**:
```python
>>> from qiskit import qasm3
>>> new_qc = qasm3.loads(qasm_string)
```

**QASM 3.0 String Format**:
```qasm
OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
h q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
```

**Key QASM 3.0 syntax**:
- `qubit[n] q;` instead of `qreg q[n];`
- `bit[n] c;` instead of `creg c[n];`
- `c[0] = measure q[0];` instead of `measure q[0] -> c[0];`

In [ ]:
def demonstrate_qasm_import():
    """
    QASM Import using qasm3.loads()
    """
    print("\n" + "="*70)
    print("QASM 3.0 IMPORT - qasm3.loads()")
    print("="*70)
    
    # QASM 3.0 string to import
    qasm_string = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
h q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
'''
    
    print("\n[1] QASM 3.0 String:")
    print(qasm_string)
    
    print("\n[2] Import with qasm3.loads()")
    print("-" * 50)
    print("new_qc = qasm3.loads(qasm_string)")
    new_qc = qasm3.loads(qasm_string)
    print("\nImported Circuit:")
    print(new_qc.draw())
    new_new_qc = qasm3.dumps(new_qc)
    print(new_new_qc)
    
    print("\n[3] Key QASM 3.0 Syntax:")
    print("-" * 50)
    print("""
    OPENQASM 3.0;              # Version declaration
    include "stdgates.inc";    # Standard gate library (not qelib1.inc!)
    qubit[2] q;                # Qubit declaration (not qreg!)
    bit[2] c;                  # Bit declaration (not creg!)
    h q[0];                    # Hadamard gate
    cx q[0], q[1];             # CNOT gate
    c[0] = measure q[0];       # Measurement (assignment style!)
    """)
    
    print("\n🎯 EXAM TIP: qasm3.loads() imports QASM 3.0 strings!")

demonstrate_qasm_import()

---

## 8.3 QASM 3.0 File Operations

**EXAM QUESTION**: How do you save/load circuits to/from QASM 3.0 files?

**ANSWER**:
```python
from qiskit import qasm3

# Save to file
with open('circuit.qasm', 'w') as f:
    qasm3.dump(qc, f)

# Load from file
loaded_qc = qasm3.load('circuit.qasm')
```

**Module functions**:
- `qasm3.dump(circuit, file)` - Write to file object
- `qasm3.load(filename)` - Read from file path
- `qasm3.dumps(circuit)` - Export to string
- `qasm3.loads(string)` - Import from string

In [ ]:
def demonstrate_qasm_file_operations():
    """
    QASM 3.0 File Operations
    """
    print("\n" + "="*70)
    print("QASM 3.0 FILE OPERATIONS")
    print("="*70)
    
    # Create circuit
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure_all()
    
    import tempfile
    import os
    
    print("\n[1] Save to File")
    print("-" * 50)
    
    # Create temp file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=False) as f:
        filename = f.name
        qasm3.dump(qc, f)
    
    print(f"Saved to: {filename}")
    print("""
# Method: qasm3.dump()
from qiskit import qasm3
with open('circuit.qasm', 'w') as f:
    qasm3.dump(qc, f)
""")
    
    print("\n[2] Load from File")
    print("-" * 50)
    loaded_qc = qasm3.load(filename)
    print("loaded_qc = qasm3.load('circuit.qasm')")
    print("\nLoaded Circuit:")
    print(loaded_qc.draw())
    
    # Cleanup
    os.unlink(filename)
    
    print("\n🎯 EXAM TIP: Use qasm3.dump() and qasm3.load() for file I/O!")

demonstrate_qasm_file_operations()

---

## 8.4 QASM 3.0 Features

**EXAM QUESTION**: What are the key features of QASM 3.0?

**ANSWER**:

| Feature | QASM 3.0 Syntax |
|---------|-----------------|
| Header | `OPENQASM 3.0;` |
| Standard library | `include "stdgates.inc";` |
| Qubit declaration | `qubit[2] q;` |
| Bit declaration | `bit[2] c;` |
| Measurement | `c[0] = measure q[0];` |
| Parameters | `input float[64] theta;` |
| Control flow | `if (c[0] == 1) { x q[1]; }` |

```python
from qiskit import qasm3

# Export to QASM 3.0
qasm3_str = qasm3.dumps(qc)

# Import from QASM 3.0
qc = qasm3.loads(qasm3_str)
```

In [ ]:
def demonstrate_qasm3_features():
    """
    QASM 3.0 Features Overview
    """
    print("\n" + "="*70)
    print("QASM 3.0 FEATURES")
    print("="*70)
    
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    print("\n[1] QASM 3.0 Export")
    print("-" * 50)
    qasm3_str = qasm3.dumps(qc)
    print("from qiskit import qasm3")
    print("qasm3_str = qasm3.dumps(qc)")
    print("\nQASM 3.0 Output:")
    print(qasm3_str)
    
    print("\n[2] Key QASM 3.0 Features")
    print("-" * 50)
    print("""
    QASM 3.0 Features:
    ✓ Modern syntax (qubit[n], bit[n])
    ✓ Assignment-style measurement: c = measure q
    ✓ Full classical control flow (if/else, for, while)
    ✓ Subroutines/functions
    ✓ Better parameter support (input keyword)
    ✓ Real-time control support
    
    Header differences:
    - OPENQASM 3.0; (not 2.0)
    - include "stdgates.inc"; (not qelib1.inc)
    """)
    
    print("\n[3] QASM 3.0 Syntax Reference")
    print("-" * 50)
    print("""
    OPENQASM 3.0;
    include "stdgates.inc";
    input float[64] theta;      // Parameters
    qubit[2] q;                  // Quantum register
    bit[2] c;                    // Classical register
    h q[0];                      // Gates
    cx q[0], q[1];
    c[0] = measure q[0];         // Measurement
    if (c[0] == 1) { x q[1]; }   // Control flow
    """)
    
    print("\n🎯 Use qasm3.dumps() and qasm3.loads() for QASM 3.0!")

demonstrate_qasm3_features()

---

## 8.5 Common QASM Patterns

**EXAM QUESTION**: What are common patterns for QASM operations?

In [ ]:
def demonstrate_common_qasm_patterns():
    """
    Common QASM 3.0 Patterns
    """
    print("\n" + "="*70)
    print("COMMON QASM 3.0 PATTERNS")
    print("="*70)
    
    print("\n[1] Round-trip: Export → Import")
    print("-" * 50)
    print("""
# Create circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Export to QASM 3.0 string
qasm_str = qasm3.dumps(qc)

# Import back (creates NEW circuit)
qc_copy = qasm3.loads(qasm_str)
""")
    
    # Demonstrate
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    qasm_str = qasm3.dumps(qc)
    qc_copy = qasm3.loads(qasm_str)
    print("Original:")
    print(qc.draw())
    print("\nCopy from QASM 3.0:")
    print(qc_copy.draw())
    
    print("\n[2] File Persistence Pattern")
    print("-" * 50)
    print("""
# Save circuit for later
with open('my_circuit.qasm', 'w') as f:
    qasm3.dump(qc, f)

# ... later ...

# Load circuit back
qc_loaded = qasm3.load('my_circuit.qasm')
""")
    
    print("\n[3] Share Circuit as String")
    print("-" * 50)
    print("""
# Export for sharing
qasm_to_share = qasm3.dumps(qc)

# Someone else imports it
qc_received = qasm3.loads(qasm_to_share)
""")

demonstrate_common_qasm_patterns()

---

## 8.6 Practical Examples

In [ ]:
def demonstrate_qasm_examples():
    """
    Practical QASM 3.0 Examples
    """
    print("\n" + "="*70)
    print("PRACTICAL QASM 3.0 EXAMPLES")
    print("="*70)
    
    print("\n[1] Bell State QASM 3.0")
    print("-" * 50)
    
    bell_qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
h q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
'''
    print(bell_qasm)
    
    # Import and display
    bell_qc = qasm3.loads(bell_qasm)
    print("Imported Circuit:")
    print(bell_qc.draw())
    
    print("\n[2] GHZ State QASM 3.0")
    print("-" * 50)
    
    ghz_qasm = '''OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
bit[3] c;
h q[0];
cx q[0], q[1];
cx q[0], q[2];
c[0] = measure q[0];
c[1] = measure q[1];
c[2] = measure q[2];
'''
    print(ghz_qasm)
    
    ghz_qc = qasm3.loads(ghz_qasm)
    print("Imported GHZ Circuit:")
    print(ghz_qc.draw())
    
    print("\n[3] Understanding QASM 3.0 Structure")
    print("-" * 50)
    print("""
    OPENQASM 3.0;              # Version declaration
    include "stdgates.inc";    # Standard gate library
    qubit[2] q;                # Quantum register (2 qubits)
    bit[2] c;                  # Classical register (2 bits)
    h q[0];                    # Hadamard on qubit 0
    cx q[0], q[1];             # CNOT with q[0] control
    c[0] = measure q[0];       # Measure q[0] to c[0]
    """)

demonstrate_qasm_examples()

---

## 📊 Quick Reference Table

| Operation | Method | Example |
|-----------|--------|---------|
| Export to string | `qasm3.dumps()` | `s = qasm3.dumps(qc)` |
| Import from string | `qasm3.loads()` | `qc = qasm3.loads(s)` |
| Export to file | `qasm3.dump()` | `qasm3.dump(qc, f)` |
| Import from file | `qasm3.load()` | `qc = qasm3.load(path)` |
| Custom exporter | `qasm3.Exporter()` | `exp = qasm3.Exporter()` |

---

## 🎓 QASM 3.0 Syntax Patterns

**Pattern 1: Export circuit to QASM 3.0**
```python
from qiskit import qasm3
qasm_string = qasm3.dumps(qc)
```

**Pattern 2: Import circuit from QASM 3.0**
```python
qc = qasm3.loads(qasm_string)
```

**Pattern 3: File I/O**
```python
# Save
with open('circuit.qasm', 'w') as f:
    qasm3.dump(qc, f)

# Load
qc = qasm3.load('circuit.qasm')
```

## 🔧 Troubleshooting Guide

```
Error: QASM syntax error
→ Check QASM 3.0 format: OPENQASM 3.0; include "stdgates.inc";

Error: gate 'xxx' not defined
→ Make sure to include "stdgates.inc" for standard gates

Error: Unexpected token
→ Verify QASM 3.0 syntax: qubit[n] instead of qreg q[n]

Error: FileNotFoundError
→ Check file path for qasm3.load()
```

### ✅ OpenQASM 3.0 Checklist

```
□ Export: Using qasm3.dumps(qc)?
□ Import: Using qasm3.loads(string)?
□ File export: Using qasm3.dump(qc, file)?
□ File import: Using qasm3.load(path)?
□ Header: OPENQASM 3.0;
□ Include: stdgates.inc (not qelib1.inc!)
□ Registers: qubit[n] and bit[n] (not qreg/creg!)
□ Measurement: c = measure q (assignment style)
```

## 💡 Key Takeaways

1. **Export**: `qasm3.dumps(qc)` - Export circuit to QASM 3.0 string
2. **Import**: `qasm3.loads(string)` - Import from QASM 3.0 string
3. **File Export**: `qasm3.dump(qc, file)` - Write to file object
4. **File Import**: `qasm3.load(path)` - Read from file path
5. **Header**: `OPENQASM 3.0;` with `include "stdgates.inc";`
6. **Syntax**: `qubit[n] q;` and `bit[n] c;` (not qreg/creg)
7. **Measurement**: `c[0] = measure q[0];` (assignment style)

---

**QASM 3.0 Key Syntax**:
- `OPENQASM 3.0;` - Version header
- `include "stdgates.inc";` - Standard gates
- `qubit[n] q;` - Quantum register
- `bit[n] c;` - Classical register  
- `c = measure q;` - Measurement
- `if (c == 1) { x q; }` - Control flow

🎯 **Remember**: QASM 3.0 uses modern syntax with assignment-style measurements!